# Performance Pandas vs. Cudf

Pandas

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
dataFrame = pd.read_csv("data/telco.csv", sep=";")
dataFrame.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
from action_rules import ActionRules
# Parameters
stable_attributes = ["gender", "SeniorCitizen", "Partner"]
flexible_attributes = ["PhoneService", 
                       "InternetService", 
                       "OnlineSecurity", 
                       "DeviceProtection", 
                       "TechSupport",
                       "StreamingTV"]
target = 'Churn'
min_stable_attributes = 1
min_flexible_attributes = 1 #min 1
min_undesired_support = 4
min_undesired_confidence = 0.6
min_desired_support = 4
min_desired_confidence = 0.6
undesired_state = 'Yes'
desired_state = 'No'
# Action Rules Mining
action_rules = ActionRules(min_stable_attributes, min_flexible_attributes, min_undesired_support, min_undesired_confidence, min_desired_support,min_desired_confidence, verbose=False)

In [4]:
action_rules.fit(
    dataFrame, 
    stable_attributes, 
    flexible_attributes, 
    target, 
    undesired_state,
    desired_state, 
    False
)

In [5]:
%timeit action_rules.fit(dataFrame, stable_attributes, flexible_attributes, target, undesired_state, desired_state, False)

3.8 s ± 55.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# Count action rules
len(action_rules.get_rules().get_ar_notation())

735

Cudf

In [8]:
import cudf

In [9]:
dataFrame = cudf.read_csv("data/telco.csv", sep=";")
dataFrame.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [10]:
from action_rules import ActionRules
# Parameters
stable_attributes = ["gender", "SeniorCitizen", "Partner"]
flexible_attributes = ["PhoneService", 
                       "InternetService", 
                       "OnlineSecurity", 
                       "DeviceProtection", 
                       "TechSupport",
                       "StreamingTV"]
target = 'Churn'
min_stable_attributes = 1
min_flexible_attributes = 1 #min 1
min_undesired_support = 4
min_undesired_confidence = 0.6
min_desired_support = 4
min_desired_confidence = 0.6
undesired_state = 'Yes'
desired_state = 'No'
# Action Rules Mining
action_rules = ActionRules(min_stable_attributes, min_flexible_attributes, min_undesired_support, min_undesired_confidence, min_desired_support,min_desired_confidence, verbose=False)

In [ ]:
action_rules.fit(
    dataFrame, 
    stable_attributes, 
    flexible_attributes, 
    target, 
    undesired_state,
    desired_state, 
    True, # GPU Support
)

In [ ]:
%timeit action_rules.fit(dataFrame, stable_attributes, flexible_attributes, target, undesired_state, desired_state, True)

In [ ]:
# Count action rules
len(action_rules.get_rules().get_ar_notation())